In [50]:
import os
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.core import Spin, Orbital
import numpy as np
from copy import deepcopy
import matplotlib as mpl
mpl.use('Agg');
from matplotlib import pyplot as plt
import argparse

In [51]:
def extractVasprun(VPRunFile):
    '''
    Extract neccessary information from vasprun.xml
    '''
    VPR=Vasprun(VPRunFile);
    TDOS,IDOS,PDOS=VPR.tdos,VPR.idos,VPR.pdos;
    Energies=TDOS.energies-TDOS.efermi;
    ProtoStr=VPR.structures[-1];
    EleSyms=list(ProtoStr.symbol_set);
    EleIndDict={Ele:list(ProtoStr.indices_from_symbol(Ele)) for Ele in EleSyms};
    NAtom=len(ProtoStr);
    return Energies, PDOS, NAtom, EleIndDict, EleSyms;

In [56]:
Energies,PDOS,NAtom,EleIndDict,EleSyms=extractVasprun('../3.gamma_band/vasprun.xml')
print(Energies)

[-4.67311572 -4.66311572 -4.65311572 ... 10.30688428 10.31688428
 10.32688428]


In [53]:
def get_atom_pdos(EleIndDict,EleSyms,Energies,PDOS,NAtom,maxsqn=3,sm=2,ELim=[-5.0,5.0]):
    '''
    Get the atomic projected density of states,
    will return the figure handle for further process
    '''
    AtomDOSColor = ['g','y','r','b'];
    NOrb=0; OrbIndsGrp=[]; Spins=[]; OrbLst=list(Orbital);
    OrbDict={0:'s',1:'p',2:'d',3:'f'};
    SpinName={0:Spin.up,1:Spin.down};
    for s in range(sm): Spins.append(SpinName[s]);

    ###Start the visualization
    PDOSArryDict={}; MaxDOS=0; MinDOS=1e10;itmp=0;
    PDOSData = np.zeros((len(Energies),2*len(EleSyms)))
    Fig=plt.figure(figsize=(4,6));
    for EleInd, EleSym in enumerate(EleSyms):
        IndLst=EleIndDict[EleSym];
        for si,s in enumerate(Spins):
            DOSArry=np.zeros(PDOS[0][OrbLst[0]][s].shape);
            for AtomInd in IndLst:
                for OrbInd,Orb in enumerate(OrbLst):
                    if si%2==0: DOSArry+=PDOS[AtomInd][Orb][s];
                    else: DOSArry-=PDOS[AtomInd][Orb][s];
            if si%2==0:
                plt.plot(DOSArry,Energies,color=AtomDOSColor[EleInd],\
                        linewidth=2.0,label=EleSym);
                Tag='{}_up'.format(EleSym);
            else:
                plt.plot(DOSArry,Energies,color=AtomDOSColor[EleInd],linewidth=2.0);
                Tag='{}_dn'.format(EleSym);
            PDOSArryDict[Tag]=DOSArry;
            PDOSData[:,itmp]=DOSArry;
            itmp = itmp + 1;
            if np.max(DOSArry[5:-5]) > MaxDOS: MaxDOS=np.max(DOSArry[5:-5]);
            if np.min(DOSArry[5:-5]) < MinDOS: MinDOS=np.min(DOSArry[5:-5]);
    ####Some logistics for visualization
    plt.xlabel('DOS (a.u.)',fontsize=21); plt.ylabel('E-E$_f$ (eV)',fontsize=21);
    plt.xlim((MinDOS*1.1,MaxDOS*1.1));
    plt.ylim((ELim[0],ELim[1]));
    plt.xticks(fontsize=18); plt.yticks(fontsize=18);
    plt.legend(fontsize=18,loc='best');
    plt.tight_layout();
    return plt, PDOSArryDict, PDOSData;

In [54]:
plt,PDOSArryDict,PDOSData=get_atom_pdos(EleIndDict,EleSyms,Energies,PDOS,NAtom,4,2,[-1.8,7.8])

In [55]:
print(2*len(EleSyms))
aa = np.zeros((3,2))
aa[:,0] = [1,2,3]
np.savetxt('test.out', PDOSData)

8
